# Image Processing Summary
Author: Melissa

This NB uses the file archive_image_list.txt, which was from all images for 2021A-0113 in the NOIRLab Data Archive.

**Class:** MLG has assigned all archive images a "class", as follows.<br>
Some images were obtained with a different exposure time and are not "standard".

`| Image Class  | filters | exposure times |     |`<br>
`| COSMOS       | g r i   | 60  86  130    | std |`<br>
`| COSMOS-MOON  | g r i   | 60  30  30     |     |`<br>
`| COSMOS-AGN   | g r i   | 80  70  90     |     |`<br>
`| ELAIS        | g r i   | 60  86  130    | std |`<br>
`| DECaPS       | g r i z | 96  50  30  30 | std |`<br>
`| DECaPS-MOON  | g r i z | -   25  30  30 |     |`<br>


## Set Up

In [ ]:
import psycopg2
import psycopg2.extras
import getpass

import os
import numpy as np
import matplotlib.pyplot as plt
from copy import deepcopy

from astropy.coordinates import SkyCoord

In [ ]:
dbuser = input("DB User: ")
dbpasswd = getpass.getpass("DB Password: ")

In [ ]:
db = psycopg2.connect(f"dbname='decat' user='{dbuser}' password='{dbpasswd}' host='decatdb.lbl.gov'")

In [ ]:
cursor = db.cursor( cursor_factory = psycopg2.extras.DictCursor )

In [ ]:
### If you want to print table schema
tables = ['exposures','subtractions']
for table in tables:
    query = "SELECT column_name, data_type FROM information_schema.columns WHERE table_name=%s"
    cursor.execute( query, ( table, ))
    print( f"\nTABLE: {table}\n===========================" )
    for row in cursor:
        print( f"{row['column_name']:24s}  :  {row['data_type']:s}" )

<br>
<br>

## Generate numpy arrays to work with

### exposures table
put selected columns into numpy arrays

In [ ]:
query = "SELECT id, ra, dec, filename, mjd, filter, proposalid, header FROM exposures"
cursor.execute( query )
results = np.array( cursor.fetchall() ).transpose()
exp_id  = np.asarray( results[0], dtype='int' )
exp_ra  = np.asarray( results[1], dtype='float' )
exp_dec = np.asarray( results[2], dtype='float' )
exp_fnm = np.asarray( results[3], dtype='str' )
exp_mjd = np.asarray( results[4], dtype='float' )
exp_fil = np.asarray( results[5], dtype='str' )
exp_pid = np.asarray( results[6], dtype='str' )
exp_hdr = np.asarray( results[7], dtype='str' )
del query, results

In [ ]:
### exp_fbs : just the filename base (without the extension)
exp_fbs = deepcopy(exp_fnm)
for i,fnm in enumerate(exp_fnm):
    exp_fbs[i] = fnm.split('.')[0]
print(exp_fnm[0])
print(exp_fbs[0])

In [ ]:
### The header is a string that can be converted to a directory
# temp = eval(exp_hdr[0])
# temp
# print("temp['AZ'] = ",temp['AZ'])

### subtractions table
put selected columns into numpy arrays

In [ ]:
query = "SELECT id, ra, dec, lmt_mg, seeing, skysig, magzp, "+ \
        "ccdnum, image_id, exposure_id FROM subtractions"
cursor.execute( query )
results = np.array( cursor.fetchall() ).transpose()
sub_id  = np.asarray( results[0], dtype='int' )
sub_ra  = np.asarray( results[1], dtype='float' )
sub_dec = np.asarray( results[2], dtype='float' )
sub_lmg = np.asarray( results[3], dtype='float' )
sub_see = np.asarray( results[4], dtype='float' )
sub_sks = np.asarray( results[5], dtype='float' )
sub_mzp = np.asarray( results[6], dtype='float' )
sub_ccd = np.asarray( results[7], dtype='int' )
sub_iid = np.asarray( results[8], dtype='int' )
sub_eid = np.asarray( results[9], dtype='int' )
del query, results

add a few more qualities to the sub_*

In [ ]:
### sub_fil : the filter of the exposure associated with the subtraction
### sub_mjd : the MJD of the exposure associated with the subtraction
### sub_off : the distance in arcsec of the ccd from the image center

sub_fil = np.zeros( len(sub_eid), dtype='str' )
sub_mjd = np.zeros( len(sub_eid), dtype='float' )
sub_era = np.zeros( len(sub_eid), dtype='float' )
sub_ede = np.zeros( len(sub_eid), dtype='float' )
for i in range(len(sub_eid)):
    tx = np.where( exp_id == sub_eid[i] )[0]
    if len(tx) == 1:
        sub_fil[i] = exp_fil[tx[0]]
        sub_mjd[i] = exp_mjd[tx[0]]
        sub_era[i] = exp_ra[tx[0]]
        sub_ede[i] = exp_dec[tx[0]]
    elif len(tx) == 0:
        print('No exposure id match for: ', sub_eid[i])
    elif len(tx) > 1:
        print('More than one exposure id match for: ', sub_eid[i])
    del tx

c1 = SkyCoord(sub_ra,sub_dec,unit="deg")
c2 = SkyCoord(sub_era,sub_ede,unit="deg")
sub_off = c1.separation(c2).arcsec

del sub_era,sub_ede,c1,c2

In [ ]:
### exp_mdn_see : median seeing over all subtractions for this exposure
### exp_mdn_lmg : median limiting magnitude over all subtractions for this exposure
### exp_mdn_sks : median skysig over all subtractions for this exposure
### exp_mdn_mzp : median zeropoint over all subtractions for this exposure

exp_mdn_see = np.zeros( len(exp_id), dtype='float' )
exp_mdn_lmg = np.zeros( len(exp_id), dtype='float' )
exp_mdn_sks = np.zeros( len(exp_id), dtype='float' )
exp_mdn_mzp = np.zeros( len(exp_id), dtype='float' )
for i in range(len(exp_id)):
    sx = np.where( sub_eid == exp_id[i] )[0]
    exp_mdn_see[i] = np.median(sub_see[sx])
    exp_mdn_lmg[i] = np.median(sub_lmg[sx])
    exp_mdn_sks[i] = np.median(sub_sks[sx])
    exp_mdn_mzp[i] = np.median(sub_mzp[sx])

### sub_rel_see : fraction of subtraction seeing / median seeing for all subs of exp
### sub_rel_lmg : " but for limiting magnitude
### sub_rel_sks : " but for skysig
### sub_rel_mzp : " but for zeropoint

sub_rel_see = np.zeros( len(sub_eid), dtype='float' )
sub_rel_lmg = np.zeros( len(sub_eid), dtype='float' )
sub_rel_sks = np.zeros( len(sub_eid), dtype='float' )
sub_rel_mzp = np.zeros( len(sub_eid), dtype='float' )
for i in range(len(sub_eid)):
    tx = np.where( exp_id == sub_eid[i] )[0]
    sub_rel_see[i] = sub_see[i] / exp_mdn_see[tx[0]]
    sub_rel_lmg[i] = sub_lmg[i] / exp_mdn_lmg[tx[0]]
    sub_rel_sks[i] = sub_sks[i] / exp_mdn_sks[tx[0]]
    sub_rel_mzp[i] = sub_mzp[i] / exp_mdn_mzp[tx[0]]

In [ ]:
### Print value ranges for the exposure id, subtraction id, subtraction ccd
# print(np.min(exp_id),np.max(exp_id))
# print(np.min(sub_ccd),np.max(sub_ccd))
# print(np.min(sub_iid),np.max(sub_iid))
# print(np.min(sub_eid),np.max(sub_eid))

In [ ]:
### Are there any bad ccds?
### Yes. CCD = 31, 61 have no subtractions
### Known issue: http://www.ctio.noao.edu/noao/node/2630

# usebins = np.arange( 63, dtype='float' ) + 0.5
# plt.hist(sub_ccd, bins=usebins, histtype='step')
# plt.xlabel('CCD Number')
# plt.ylabel('Number of Subtractions')

### Print number of subtractions per CCD as a check
# for i in range(63):
#     tx = np.where( sub_ccd == i )[0]
#     print(i, len(tx))
#     del tx

In [ ]:
### Show map of RA, Dec of the COSMOS and ELAIS subtractions
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
cx = np.where( (sub_ra>145) & (sub_ra<155) )[0]
plt.plot( sub_ra[cx], sub_dec[cx], 'o', ms=10, alpha=0.01, color='dodgerblue')
plt.subplot(1, 2, 2)
ex = np.where( (sub_ra>2.0) & (sub_ra<12.0) )[0]
plt.plot( sub_ra[ex], sub_dec[ex], 'o', ms=10, alpha=0.01, color='darkviolet')

### archive image list
read it into numpy arrays

Melissa made this list of all 2021A-0113 images from the NOIRLab Data Archive.

* fbase : base of archive filename (without extension)
* ra, dec : coordinates in decimal degrees
* mjd : modified julian date of observation (at exposure start)
* filt : filter
* airm : airmass
* obj : object name
* expt : exposure time
* seqid : sequence id (relates to JSON file for observing)
* **class : COSMOS, COSMOS-AGN, COSMOS-MOON, ELAIS, DECaPS, DECaPS-MOON, or NONE**
   * -AGN and -MOON indicate non-standard exposure times
   * class = COSMOS, ELAIS, or DECaPS are considered "standard"
* ftype : field type, EGAL, GAL, or NONE
* moonsep : moon separation in degrees
* moonill : moon illumination as a fraction
* mpval : median image pixel value for images with moonsep<60 and moonill>0.2

In [ ]:
fnm = 'archive_image_list.txt'
arch_fbase   = np.loadtxt( fnm, dtype='str', usecols=(0) )
arch_ra      = np.loadtxt( fnm, dtype='float', usecols=(1) )
arch_dec     = np.loadtxt( fnm, dtype='float', usecols=(2) )
arch_mjd     = np.loadtxt( fnm, dtype='float', usecols=(3) )
arch_filt    = np.loadtxt( fnm, dtype='str', usecols=(4) )
arch_airm    = np.loadtxt( fnm, dtype='float', usecols=(5) )
arch_obj     = np.loadtxt( fnm, dtype='str', usecols=(6) )
arch_expt    = np.loadtxt( fnm, dtype='float', usecols=(7) )
arch_seqid   = np.loadtxt( fnm, dtype='str', usecols=(8) )
arch_class   = np.loadtxt( fnm, dtype='str', usecols=(9) )
arch_ftype   = np.loadtxt( fnm, dtype='str', usecols=(10) )
arch_moonsep = np.loadtxt( fnm, dtype='float', usecols=(11) )
arch_moonill = np.loadtxt( fnm, dtype='float', usecols=(12) )
arch_mpval   = np.loadtxt( fnm, dtype='float', usecols=(13) )

#### match archive image list to the exposures and subtractions table

In [ ]:
### exp_aid : the index of the exposure in the archive table
### exp_std : if the archive image has a "standard" classification (COSMOS, ELAIS, or DECaPS only)
exp_aid = np.zeros( len(exp_id), dtype='int' )
exp_std = np.zeros( len(exp_id), dtype='int' )

### arch_nem : number of exposure table matches
### arch_eid : expsoure id of match (first, if multiple)
arch_nem = np.zeros( len(arch_fbase), dtype='int' )
arch_eid = np.zeros( len(arch_fbase), dtype='int' )

for i,fbase in enumerate(arch_fbase):
    tx = np.where( fbase == exp_fbs )[0]
    arch_nem[i] = len(tx)
    if len(tx) == 0:
        arch_eid[i] = -1
    elif len(tx) >= 1:
        arch_eid[i] = exp_id[tx[0]]
        exp_aid[tx[0]] = i
        if (arch_class[i] == 'COSMOS') | \
        (arch_class[i] == 'ELAIS') | \
        (arch_class[i] == 'DECaPS'):
            exp_std[tx[0]] = 1
    del tx

### sub_expt    : exposure time of the image for this subtraction
### sub_moonsep : moon separation in degrees of the image for this subtraction
### sub_moonill : moon illumination (fraction) of the image for this subtraction
### sub_std     : is a standard image (COSMOS, ELAIS, or DECapS; no -AGN, no -MOON)
sub_expt    = np.zeros( len(sub_eid), dtype='float' )
sub_moonsep = np.zeros( len(sub_eid), dtype='float' )
sub_moonill = np.zeros( len(sub_eid), dtype='float' )
sub_mpixval = np.zeros( len(sub_eid), dtype='float' )
sub_std     = np.zeros( len(sub_eid), dtype='int' )
for i in range(len(sub_eid)):
    tx = np.where( exp_id == sub_eid[i] )[0]
    archid = exp_aid[tx[0]]
    sub_expt[i] = arch_expt[archid]
    sub_moonsep[i] = arch_moonsep[archid]
    sub_moonill[i] = arch_moonill[archid]
    sub_mpixval[i] = arch_mpval[archid]
    sub_std[i] = exp_std[tx[0]]

<br>
<br>

## Account for all images

#### number and classes of archive images which matched to exposure table

In [ ]:
print('N, number of archive images with N matches to exposure table:')
for i in range(4):
    tx = np.where( arch_nem == i )[0]
    print(i, len(tx))
    del tx
print('')
print('Image "class", number of archive images without a match to the exposure table.')
tx = np.where( arch_nem == 0 )[0]
tmps = np.asarray( arch_class[tx], dtype='str')
utmps = np.unique( np.sort( tmps ) )
for u,utmp in enumerate(utmps):
    tx = np.where( tmps == utmp )[0]
    print('%-12s %4i' % (utmp, len(tx)))
    del tx
print('')
print('Most of the unmatched archive images are DECaPS (as expected).')
print('The NONE image class is standard stars, pointing checks, etc.')
del tmps,utmps

#### how many exposures do not have any subtractions

In [ ]:
### Number of subtractions per exposure
exp_nsubs = np.zeros( len(exp_id), dtype='int' )
for i,expid in enumerate(exp_id):
    tx = np.where( sub_eid == expid )[0]
    exp_nsubs[i] = len(tx)
    del tx

print('Number of subtractions, number of exposures with N subtractions.')
tmps = np.sort(exp_nsubs)
utmps = np.unique(tmps)
for u,utmp in enumerate(utmps):
    tx = np.where( tmps == utmp )[0]
    print(utmp,len(tx))
    del tx

print('')
print('Most exposures have either 60 subtractions, or none; as expected.')

#### for exposures without subtractions, what type of image are they?

In [ ]:
ex = np.where( exp_nsubs == 0 )[0]

print('archive image "class", number of exposures without subtractions')
nosub_class = arch_class[ exp_aid[ex] ]
tmps = np.sort( nosub_class )
utmps = np.unique( tmps )
for u,utmp in enumerate(utmps):
    tx = np.where( tmps == utmp )[0]
    print('%-12s %4i' % (utmp,len(tx)))
    del tx
    
print('')
print('filter, number of exposures without subtractions')
nosub_fil = exp_fil[ex]
tmps = np.sort( nosub_fil )
utmps = np.unique( tmps )
for u,utmp in enumerate(utmps):
    tx = np.where( tmps == utmp )[0]
    print(utmp,len(tx))
    del tx

print('')
# Before Aug 11
# print('The exposures without subtractions are primarily the COSMOS i-band, as expected.')
# After Aug 11
print('There are almost no exposures without subtractions.')

<br>
<br>

## Plot histograms of image characteristics

Only for subtractions of 'standard' exposures/images, that have a class of COSMOS, ELAIS (or DECaPS, if there were any).
This removes the -AGN and -MOON exposures which have different (shorter) exposure times.

`| COSMOS       | g r i   | 60  86  130    | std |`<br>
`| COSMOS MOON  | g r i   | 60  30  30     |     |`<br>
`| COSMOS AGN   | g r i   | 80  70  90     |     |`<br>
`| ELAIS        | g r i   | 60  86  130    | std |`<br>
`| DECaPS       | g r i z | 96  50  30  30 | std |`<br>
`| DECaPS MOON  | g r i z | -   25  30  30 |     |`<br>

I.e., only use subtractions with `sub_std == 1`.

In [ ]:
### COSMOS subtractions in g and r with standard exposure times
cxg = np.where( (sub_ra>145) & (sub_ra<155)  & (sub_fil == 'g') & (sub_std == 1) )[0]
cxr = np.where( (sub_ra>145) & (sub_ra<155)  & (sub_fil == 'r') & (sub_std == 1) )[0]
cxi = np.where( (sub_ra>145) & (sub_ra<155)  & (sub_fil == 'i') & (sub_std == 1) )[0]

### ELAIS subtractions in g and r with standard exposure times
exg = np.where( (sub_ra>2.0) & (sub_ra<12.0) & (sub_fil == 'g') & (sub_std == 1) )[0]
exr = np.where( (sub_ra>2.0) & (sub_ra<12.0) & (sub_fil == 'r') & (sub_std == 1) )[0]
exi = np.where( (sub_ra>2.0) & (sub_ra<12.0) & (sub_fil == 'i') & (sub_std == 1) )[0]

# ### **IF* you want to do this for all images, not just the 'standard' exposure times:
# ### COSMOS subtractions in g and r with standard exposure times
# cxg = np.where( (sub_ra>145) & (sub_ra<155)  & (sub_fil == 'g') )[0]
# cxr = np.where( (sub_ra>145) & (sub_ra<155)  & (sub_fil == 'r') )[0]
# ### ELAIS subtractions in g and r with standard exposure times
# exg = np.where( (sub_ra>2.0) & (sub_ra<12.0) & (sub_fil == 'g') )[0]
# exr = np.where( (sub_ra>2.0) & (sub_ra<12.0) & (sub_fil == 'r') )[0]
# exi = np.where( (sub_ra>2.0) & (sub_ra<12.0) & (sub_fil == 'i') )[0]

In [ ]:
plt.figure(figsize=(12, 4))
plt.rcParams.update({'font.size': 13})
# plt.subplots(1, 3, sharey=True)
ax1 = plt.subplot(1, 3, 1)
plt.hist( [sub_lmg[cxg],sub_lmg[exg]], bins=30, stacked=True, color=['darkgreen','limegreen'], label=['COSMOS','ELAIS'])
plt.xlabel('g-band Limiting Magnitude [mag]')
plt.ylabel('Number of Processed CCDs')
plt.legend(loc='upper left')
ax2 = plt.subplot(1, 3, 2, sharex=ax1, sharey=ax1)
plt.hist( [sub_lmg[cxr],sub_lmg[exr]], bins=30, stacked=True, color=['red','darkorange'], label=['COSMOS','ELAIS'])
plt.xlabel('r-band Limiting Magnitude [mag]')
plt.legend(loc='upper left')
ax3 = plt.subplot(1, 3, 3, sharex=ax1, sharey=ax1)
plt.hist( [sub_lmg[cxi],sub_lmg[exi]], bins=30, stacked=True, color=['brown','peru'], label=['COSMOS','ELAIS'])
plt.xlabel('i-band Limiting Magnitude [mag]')
plt.legend(loc='upper left')
plt.tight_layout(pad=1.0)
plt.savefig('image_processing_summary_figures/limmag_dist')
plt.show()

plt.figure(figsize=(12, 4))
ax1 = plt.subplot(1, 3, 1)
plt.hist( [sub_see[cxg],sub_see[exg]], bins=30, stacked=True, color=['darkgreen','limegreen'], label=['COSMOS','ELAIS'])
plt.xlabel('g-band Seeing (arcsec)')
plt.ylabel('Number of Processed CCDs')
plt.legend(loc='upper right')
ax2 = plt.subplot(1, 3, 2, sharex=ax1, sharey=ax1)
plt.hist( [sub_see[cxr],sub_see[exr]], bins=30, stacked=True, color=['red','darkorange'], label=['COSMOS','ELAIS'])
plt.xlabel('r-band Seeing (arcsec)')
plt.legend(loc='upper right')
ax3 = plt.subplot(1, 3, 3, sharex=ax1, sharey=ax1)
plt.hist( [sub_see[cxi],sub_see[exi]], bins=30, stacked=True, color=['brown','peru'], label=['COSMOS','ELAIS'])
plt.xlabel('i-band Seeing (arcsec)')
plt.legend(loc='upper right')
plt.tight_layout(pad=1.0)
plt.savefig('image_processing_summary_figures/seeing_dist')
plt.show()

plt.figure(figsize=(12, 4))
ax1 = plt.subplot(1, 3, 1)
plt.hist( [np.log10(sub_sks[cxg]),np.log10(sub_sks[exg])], bins=30, stacked=True, color=['darkgreen','limegreen'], label=['COSMOS','ELAIS'])
plt.xlabel('g-band Log(sigma_sky) [log(counts)]')
plt.ylabel('Number of Processed CCDs')
plt.legend(loc='upper right')
ax2 = plt.subplot(1, 3, 2, sharex=ax1, sharey=ax1)
plt.hist( [np.log10(sub_sks[cxr]),np.log10(sub_sks[exr])], bins=30, stacked=True, color=['red','darkorange'], label=['COSMOS','ELAIS'])
plt.xlabel('r-band Log(sigma_sky) [log(counts)]')
plt.legend(loc='upper right')
ax3 = plt.subplot(1, 3, 3, sharex=ax1, sharey=ax1)
plt.hist( [np.log10(sub_sks[cxi]),np.log10(sub_sks[exi])], bins=30, stacked=True, color=['brown','peru'], label=['COSMOS','ELAIS'])
plt.xlabel('i-band Log(sigma_sky) [log(counts)]')
plt.legend(loc='upper right')
plt.tight_layout(pad=1.0)
plt.savefig('image_processing_summary_figures/logsky_dist')
plt.show()

plt.figure(figsize=(12, 4))
ax1 = plt.subplot(1, 3, 1)
plt.hist( [sub_mzp[cxg],sub_mzp[exg]], bins=30, stacked=True, color=['darkgreen','limegreen'], label=['COSMOS','ELAIS'])
plt.xlabel('Zeropoint (g)')
plt.ylabel('Number of Processed CCDs')
plt.legend(loc='upper left')
ax2 = plt.subplot(1, 3, 2, sharex=ax1, sharey=ax1)
plt.hist( [sub_mzp[cxr],sub_mzp[exr]], bins=30, stacked=True, color=['red','darkorange'], label=['COSMOS','ELAIS'])
plt.xlabel('Zeropoint (r)')
plt.legend(loc='upper left')
ax3 = plt.subplot(1, 3, 3, sharex=ax1, sharey=ax1)
plt.hist( [sub_mzp[cxi],sub_mzp[exi]], bins=30, stacked=True, color=['brown','peru'], label=['COSMOS','ELAIS'])
plt.xlabel('Zeropoint (i)')
plt.legend(loc='upper left')
plt.tight_layout(pad=1.0)
plt.savefig('image_processing_summary_figures/zeropoing_dist')
plt.show()

<br>
<br>

## Plot correlations between IQ parameters

### limiting magnitude, seeing, log10(skysig), and zeropoint VS. MJD

In [ ]:
plt.figure(figsize=(12, 3))
plt.subplot(1, 3, 1)
plt.plot( sub_mjd[cxg], sub_lmg[cxg], 'o', ms=2, alpha=0.1, color='darkgreen', label='COSMOS')
plt.plot( sub_mjd[exg], sub_lmg[exg], 'o', ms=2, alpha=0.1, color='limegreen', label='ELAIS')
plt.xlabel('g-band MJD')
plt.ylabel('g-band lmt_mg')
# plt.legend(loc='upper left')
plt.subplot(1, 3, 2)
plt.plot( sub_mjd[cxr], sub_lmg[cxr], 'o', ms=2, alpha=0.1, color='red', label='COSMOS')
plt.plot( sub_mjd[exr], sub_lmg[exr], 'o', ms=2, alpha=0.1, color='darkorange', label='ELAIS')
plt.xlabel('r-band MJD')
plt.ylabel('r-band lmg_mg')
# plt.legend(loc='upper left')
plt.subplot(1, 3, 3)
plt.plot( sub_mjd[cxi], sub_lmg[cxi], 'o', ms=2, alpha=0.1, color='brown', label='COSMOS')
plt.plot( sub_mjd[exi], sub_lmg[exi], 'o', ms=2, alpha=0.1, color='peru', label='ELAIS')
plt.xlabel('i-band MJD')
# plt.ylabel('i-band lmg_mg')
plt.show()

plt.figure(figsize=(12, 3))
plt.subplot(1, 3, 1)
plt.plot( sub_mjd[cxg], sub_see[cxg], 'o', ms=2, alpha=0.1, color='darkgreen', label='COSMOS')
plt.plot( sub_mjd[exg], sub_see[exg], 'o', ms=2, alpha=0.1, color='limegreen', label='ELAIS')
plt.xlabel('g-band MJD')
plt.ylabel('g-band seeing')
# plt.legend(loc='upper left')
plt.subplot(1, 3, 2)
plt.plot( sub_mjd[cxr], sub_see[cxr], 'o', ms=2, alpha=0.1, color='red', label='COSMOS')
plt.plot( sub_mjd[exr], sub_see[exr], 'o', ms=2, alpha=0.1, color='darkorange', label='ELAIS')
plt.xlabel('r-band MJD')
# plt.ylabel('r-band seeing')
# plt.legend(loc='upper left')
plt.subplot(1, 3, 3)
plt.plot( sub_mjd[cxi], sub_see[cxi], 'o', ms=2, alpha=0.1, color='brown', label='COSMOS')
plt.plot( sub_mjd[exi], sub_see[exi], 'o', ms=2, alpha=0.1, color='peru', label='ELAIS')
plt.xlabel('i-band MJD')
# plt.ylabel('i-band seeing')
plt.show()

plt.figure(figsize=(12, 3))
plt.subplot(1, 3, 1)
plt.plot( sub_mjd[cxg], np.log10(sub_sks[cxg]), 'o', ms=2, alpha=0.1, color='darkgreen', label='COSMOS')
plt.plot( sub_mjd[exg], np.log10(sub_sks[exg]), 'o', ms=2, alpha=0.1, color='limegreen', label='ELAIS')
plt.xlabel('g-band MJD')
plt.ylabel('log10(g-band skysig)')
# plt.legend(loc='upper left')
plt.subplot(1, 3, 2)
plt.plot( sub_mjd[cxr], np.log10(sub_sks[cxr]), 'o', ms=2, alpha=0.1, color='red', label='COSMOS')
plt.plot( sub_mjd[exr], np.log10(sub_sks[exr]), 'o', ms=2, alpha=0.1, color='darkorange', label='ELAIS')
plt.xlabel('r-band MJD')
# plt.ylabel('r-band skysig')
# plt.legend(loc='upper left')
plt.subplot(1, 3, 3)
plt.plot( sub_mjd[cxi], np.log10(sub_sks[cxi]), 'o', ms=2, alpha=0.1, color='brown', label='COSMOS')
plt.plot( sub_mjd[exi], np.log10(sub_sks[exi]), 'o', ms=2, alpha=0.1, color='peru', label='ELAIS')
plt.xlabel('i-band MJD')
# plt.ylabel('i-band skysig')
plt.show()

plt.figure(figsize=(12, 3))
plt.subplot(1, 3, 1)
plt.plot( sub_mjd[cxg], sub_mzp[cxg], 'o', ms=2, alpha=0.1, color='darkgreen', label='COSMOS')
plt.plot( sub_mjd[exg], sub_mzp[exg], 'o', ms=2, alpha=0.1, color='limegreen', label='ELAIS')
plt.xlabel('g-band MJD')
plt.ylabel('g-band mag_zp')
# plt.legend(loc='upper left')
plt.subplot(1, 3, 2)
plt.plot( sub_mjd[cxr], sub_mzp[cxr], 'o', ms=2, alpha=0.1, color='red', label='COSMOS')
plt.plot( sub_mjd[exr], sub_mzp[exr], 'o', ms=2, alpha=0.1, color='darkorange', label='ELAIS')
plt.xlabel('r-band MJD')
# plt.ylabel('r-band mag_zp')
# plt.legend(loc='upper left')
plt.subplot(1, 3, 3)
plt.plot( sub_mjd[cxi], sub_mzp[cxi], 'o', ms=2, alpha=0.1, color='brown', label='COSMOS')
plt.plot( sub_mjd[exi], sub_mzp[exi], 'o', ms=2, alpha=0.1, color='peru', label='ELAIS')
plt.xlabel('i-band MJD')
# plt.ylabel('i-band mag_zp')
plt.show()

### magzp VS. limiting magnitude, seeing, and log10(skysig)

In [ ]:
plt.figure(figsize=(12, 3))
plt.subplot(1, 3, 1)
plt.plot( sub_lmg[cxg], sub_mzp[cxg], 'o', ms=2, alpha=0.1, color='darkgreen', label='COSMOS')
plt.plot( sub_lmg[exg], sub_mzp[exg], 'o', ms=2, alpha=0.1, color='limegreen', label='ELAIS')
plt.xlabel('g-band lmt_mg')
plt.ylabel('g-band mag_zp')
# plt.legend(loc='upper left')
plt.subplot(1, 3, 2)
plt.plot( sub_lmg[cxr], sub_mzp[cxr], 'o', ms=2, alpha=0.1, color='red', label='COSMOS')
plt.plot( sub_lmg[exr], sub_mzp[exr], 'o', ms=2, alpha=0.1, color='darkorange', label='ELAIS')
plt.xlabel('r-band lmt_mg')
# plt.ylabel('r-band mag_zp')
# plt.legend(loc='upper left')
plt.subplot(1, 3, 3)
plt.plot( sub_lmg[cxi], sub_mzp[cxi], 'o', ms=2, alpha=0.1, color='brown', label='COSMOS')
plt.plot( sub_lmg[exi], sub_mzp[exi], 'o', ms=2, alpha=0.1, color='peru', label='ELAIS')
plt.xlabel('i-band lmt_mg')
# plt.ylabel('i-band mag_zp')
plt.show()

plt.figure(figsize=(12, 3))
plt.subplot(1, 3, 1)
plt.plot( sub_see[cxg], sub_mzp[cxg], 'o', ms=2, alpha=0.1, color='darkgreen', label='COSMOS')
plt.plot( sub_see[exg], sub_mzp[exg], 'o', ms=2, alpha=0.1, color='limegreen', label='ELAIS')
plt.xlabel('g-band seeing')
plt.ylabel('g-band mag_zp')
# plt.legend(loc='upper left')
plt.subplot(1, 3, 2)
plt.plot( sub_see[cxr], sub_mzp[cxr], 'o', ms=2, alpha=0.1, color='red', label='COSMOS')
plt.plot( sub_see[exr], sub_mzp[exr], 'o', ms=2, alpha=0.1, color='darkorange', label='ELAIS')
plt.xlabel('r-band seeing')
# plt.ylabel('r-band mag_zp')
# plt.legend(loc='upper left')
plt.subplot(1, 3, 3)
plt.plot( sub_see[cxi], sub_mzp[cxi], 'o', ms=2, alpha=0.1, color='brown', label='COSMOS')
plt.plot( sub_see[exi], sub_mzp[exi], 'o', ms=2, alpha=0.1, color='peru', label='ELAIS')
plt.xlabel('i-band seeing')
# plt.ylabel('i-band mag_zp')
plt.show()

plt.figure(figsize=(12, 3))
plt.subplot(1, 3, 1)
plt.plot( np.log10(sub_sks[cxg]), sub_mzp[cxg], 'o', ms=2, alpha=0.1, color='darkgreen', label='COSMOS')
plt.plot( np.log10(sub_sks[exg]), sub_mzp[exg], 'o', ms=2, alpha=0.1, color='limegreen', label='ELAIS')
plt.xlabel('log10(g-band skysig)')
plt.ylabel('g-band mag_zp')
# plt.legend(loc='upper left')
plt.subplot(1, 3, 2)
plt.plot( np.log10(sub_sks[cxr]), sub_mzp[cxr], 'o', ms=2, alpha=0.1, color='red', label='COSMOS')
plt.plot( np.log10(sub_sks[exr]), sub_mzp[exr], 'o', ms=2, alpha=0.1, color='darkorange', label='ELAIS')
plt.xlabel('log10(r-band skysig)')
# plt.ylabel('r-band mag_zp')
# plt.legend(loc='upper left')
plt.subplot(1, 3, 3)
plt.plot( np.log10(sub_sks[cxi]), sub_mzp[cxi], 'o', ms=2, alpha=0.1, color='brown', label='COSMOS')
plt.plot( np.log10(sub_sks[exi]), sub_mzp[exi], 'o', ms=2, alpha=0.1, color='peru', label='ELAIS')
plt.xlabel('log10(i-band skysig)')
# plt.ylabel('i-band mag_zp')
plt.show()


### seeing and log10(skysig) VS. limiting magnitude

In [ ]:
plt.figure(figsize=(12, 3))
plt.rcParams.update({'font.size': 13})
ax1 = plt.subplot(1, 3, 1)
plt.plot( sub_see[cxg], sub_lmg[cxg], 'o', ms=2, alpha=0.1, color='darkgreen', label='COSMOS')
plt.plot( sub_see[exg], sub_lmg[exg], 'o', ms=2, alpha=0.1, color='limegreen', label='ELAIS')
plt.xlabel('g-band Seeing [arcsec]')
plt.ylabel('Limiting Magnitude [mag]')
# plt.legend(loc='upper left')
ax2 = plt.subplot(1, 3, 2, sharex=ax1, sharey=ax1)
plt.plot( sub_see[cxr], sub_lmg[cxr], 'o', ms=2, alpha=0.1, color='red', label='COSMOS')
plt.plot( sub_see[exr], sub_lmg[exr], 'o', ms=2, alpha=0.1, color='darkorange', label='ELAIS')
plt.xlabel('r-band Seeing [arcsec]')
# plt.ylabel('r-band lmt_mg')
# plt.legend(loc='upper left')
ax3 = plt.subplot(1, 3, 3, sharex=ax1, sharey=ax1)
plt.plot( sub_see[cxi], sub_lmg[cxi], 'o', ms=2, alpha=0.1, color='brown', label='COSMOS')
plt.plot( sub_see[exi], sub_lmg[exi], 'o', ms=2, alpha=0.1, color='peru', label='ELAIS')
plt.xlabel('i-band Seeing [arcsec]')
# plt.ylabel('i-band lmt_mg')
plt.tight_layout(pad=0.7)
plt.savefig('image_processing_summary_figures/seeing_vs_limmag')
plt.show()

plt.figure(figsize=(12, 3))
plt.rcParams.update({'font.size': 13})
ax1 = plt.subplot(1, 3, 1)
plt.plot( np.log10(sub_sks[cxg]), sub_lmg[cxg], 'o', ms=2, alpha=0.1, color='darkgreen', label='COSMOS')
plt.plot( np.log10(sub_sks[exg]), sub_lmg[exg], 'o', ms=2, alpha=0.1, color='limegreen', label='ELAIS')
plt.xlabel('g-band Log(sigma_sky) [log(counts)]')
plt.ylabel('Limiting Magnitude [mag]')
# plt.legend(loc='upper left')
ax2 = plt.subplot(1, 3, 2, sharex=ax1, sharey=ax1)
plt.plot( np.log10(sub_sks[cxr]), sub_lmg[cxr], 'o', ms=2, alpha=0.1, color='red', label='COSMOS')
plt.plot( np.log10(sub_sks[exr]), sub_lmg[exr], 'o', ms=2, alpha=0.1, color='darkorange', label='ELAIS')
plt.xlabel('r-band Log(sigma_sky) [log(counts)]')
# plt.ylabel('r-band lmt_mg')
# plt.legend(loc='upper left')
ax3 = plt.subplot(1, 3, 3, sharex=ax1, sharey=ax1)
plt.plot( np.log10(sub_sks[cxi]), sub_lmg[cxi], 'o', ms=2, alpha=0.1, color='brown', label='COSMOS')
plt.plot( np.log10(sub_sks[exi]), sub_lmg[exi], 'o', ms=2, alpha=0.1, color='peru', label='ELAIS')
plt.xlabel('i-band Log(sigma_sky) [log(counts)]')
# plt.ylabel('i-band lmt_mg')
plt.tight_layout(pad=0.7)
plt.savefig('image_processing_summary_figures/skysig_vs_limmag')
plt.show()


plt.figure(figsize=(12, 3))
plt.rcParams.update({'font.size': 13})
ax1 = plt.subplot(1, 3, 1)
plt.plot( sub_moonsep[cxg]+(2.5*np.random.rand(len(cxg))), sub_lmg[cxg], 'o', ms=2, alpha=0.1, color='darkgreen', label='COSMOS')
plt.plot( sub_moonsep[exg]+(2.5*np.random.rand(len(exg))), sub_lmg[exg], 'o', ms=2, alpha=0.1, color='limegreen', label='ELAIS')
plt.xlabel('Moon Separation [deg]')
plt.ylabel('Limiting Magnitude [mag]')
# plt.legend(loc='upper left')
ax2 = plt.subplot(1, 3, 2, sharex=ax1, sharey=ax1)
plt.plot( sub_moonsep[cxr]+(2.5*np.random.rand(len(cxr))), sub_lmg[cxr], 'o', ms=2, alpha=0.1, color='red', label='COSMOS')
plt.plot( sub_moonsep[exr]+(2.5*np.random.rand(len(exr))), sub_lmg[exr], 'o', ms=2, alpha=0.1, color='darkorange', label='ELAIS')
plt.xlabel('Moon Separation [deg]')
# plt.ylabel('r-band lmt_mg')
# plt.legend(loc='upper left')
ax3 = plt.subplot(1, 3, 3, sharex=ax1, sharey=ax1)
plt.plot( sub_moonsep[cxi]+(2.5*np.random.rand(len(cxi))), sub_lmg[cxi], 'o', ms=2, alpha=0.1, color='brown', label='COSMOS')
plt.plot( sub_moonsep[exi]+(2.5*np.random.rand(len(exi))), sub_lmg[exi], 'o', ms=2, alpha=0.1, color='peru', label='ELAIS')
plt.xlabel('Moon Separation [deg]')
# plt.ylabel('i-band lmt_mg')
plt.tight_layout(pad=0.7)
plt.savefig('image_processing_summary_figures/moonsep_vs_limmag')
plt.show()

plt.figure(figsize=(12, 3))
plt.rcParams.update({'font.size': 13})
ax1 = plt.subplot(1, 3, 1)
plt.plot( sub_moonill[cxg]+(0.025*np.random.rand(len(cxg))), sub_lmg[cxg], 'o', ms=2, alpha=0.1, color='darkgreen', label='COSMOS')
plt.plot( sub_moonill[exg]+(0.025*np.random.rand(len(exg))), sub_lmg[exg], 'o', ms=2, alpha=0.1, color='limegreen', label='ELAIS')
plt.xlabel('Moon Illumination Fraction')
plt.ylabel('Limiting Magnitude [mag]')
# plt.legend(loc='upper left')
ax2 = plt.subplot(1, 3, 2, sharex=ax1, sharey=ax1)
plt.plot( sub_moonill[cxr]+(0.025*np.random.rand(len(cxr))), sub_lmg[cxr], 'o', ms=2, alpha=0.1, color='red', label='COSMOS')
plt.plot( sub_moonill[exr]+(0.025*np.random.rand(len(exr))), sub_lmg[exr], 'o', ms=2, alpha=0.1, color='darkorange', label='ELAIS')
plt.xlabel('Moon Illumination Fraction')
# plt.ylabel('r-band lmt_mg')
# plt.legend(loc='upper left')
ax3 = plt.subplot(1, 3, 3, sharex=ax1, sharey=ax1)
plt.plot( sub_moonill[cxi]+(0.025*np.random.rand(len(cxi))), sub_lmg[cxi], 'o', ms=2, alpha=0.1, color='brown', label='COSMOS')
plt.plot( sub_moonill[exi]+(0.025*np.random.rand(len(exi))), sub_lmg[exi], 'o', ms=2, alpha=0.1, color='peru', label='ELAIS')
plt.xlabel('Moon Illumination Fraction')
# plt.ylabel('i-band lmt_mg')
plt.tight_layout(pad=0.7)
plt.savefig('image_processing_summary_figures/moonill_vs_limmag')
plt.show()

### seeing and limiting magnitude VS. ccd offset from center of image

In [ ]:
plt.figure(figsize=(12, 3))
plt.subplot(1, 3, 1)
plt.plot( sub_off[cxg], sub_rel_lmg[cxg], 'o', ms=2, alpha=0.1, color='darkgreen', label='COSMOS')
plt.plot( sub_off[exg], sub_rel_lmg[exg], 'o', ms=2, alpha=0.1, color='limegreen', label='ELAIS')
plt.xlabel('g-band ccd offset (arcsec)')
plt.ylabel('g-band sub lmt_mg / exp lmt_mg')
# plt.legend(loc='upper left')
plt.subplot(1, 3, 2)
plt.plot( sub_off[cxr], sub_rel_lmg[cxr], 'o', ms=2, alpha=0.1, color='red', label='COSMOS')
plt.plot( sub_off[exr], sub_rel_lmg[exr], 'o', ms=2, alpha=0.1, color='darkorange', label='ELAIS')
plt.xlabel('r-band ccd offset (arcsec)')
# plt.ylabel('r-band sub lmt_mg / exp lmt_mg')
# plt.legend(loc='upper left')
plt.subplot(1, 3, 3)
plt.plot( sub_off[cxi], sub_rel_lmg[cxi], 'o', ms=2, alpha=0.1, color='brown', label='COSMOS')
plt.plot( sub_off[exi], sub_rel_lmg[exi], 'o', ms=2, alpha=0.1, color='peru', label='ELAIS')
plt.xlabel('i-band ccd offset (arcsec)')
# plt.ylabel('i-band sub lmt_mg / exp lmt_mg')
plt.show()

plt.figure(figsize=(12, 3))
plt.subplot(1, 3, 1)
plt.plot( sub_off[cxg], sub_rel_see[cxg], 'o', ms=2, alpha=0.1, color='darkgreen', label='COSMOS')
plt.plot( sub_off[exg], sub_rel_see[exg], 'o', ms=2, alpha=0.1, color='limegreen', label='ELAIS')
plt.xlabel('g-band ccd offset (arcsec)')
plt.ylabel('g-band sub seeing / exp seeing')
# plt.legend(loc='upper left')
plt.subplot(1, 3, 2)
plt.plot( sub_off[cxr], sub_rel_see[cxr], 'o', ms=2, alpha=0.1, color='red', label='COSMOS')
plt.plot( sub_off[exr], sub_rel_see[exr], 'o', ms=2, alpha=0.1, color='darkorange', label='ELAIS')
plt.xlabel('r-band ccd offset (arcsec)')
# plt.ylabel('r-band sub seeing / exp seeing')
# plt.legend(loc='upper left')
plt.subplot(1, 3, 3)
plt.plot( sub_off[cxi], sub_rel_see[cxi], 'o', ms=2, alpha=0.1, color='brown', label='COSMOS')
plt.plot( sub_off[exi], sub_rel_see[exi], 'o', ms=2, alpha=0.1, color='peru', label='ELAIS')
plt.xlabel('i-band ccd offset (arcsec)')
# plt.ylabel('i-band sub seeing / exp seeing')
plt.show()

plt.figure(figsize=(12, 3))
plt.subplot(1, 3, 1)
plt.plot( sub_off[cxg], sub_rel_sks[cxg], 'o', ms=2, alpha=0.1, color='darkgreen', label='COSMOS')
plt.plot( sub_off[exg], sub_rel_sks[exg], 'o', ms=2, alpha=0.1, color='limegreen', label='ELAIS')
plt.xlabel('g-band ccd offset (arcsec)')
plt.ylabel('g-band sub skysig / exp skysig')
# plt.legend(loc='upper left')
plt.subplot(1, 3, 2)
plt.plot( sub_off[cxr], sub_rel_sks[cxr], 'o', ms=2, alpha=0.1, color='red', label='COSMOS')
plt.plot( sub_off[exr], sub_rel_sks[exr], 'o', ms=2, alpha=0.1, color='darkorange', label='ELAIS')
plt.xlabel('r-band ccd offset (arcsec)')
# plt.ylabel('r-band sub skysig / exp skysig')
# plt.legend(loc='upper left')
plt.subplot(1, 3, 3)
plt.plot( sub_off[cxi], sub_rel_sks[cxi], 'o', ms=2, alpha=0.1, color='brown', label='COSMOS')
plt.plot( sub_off[exi], sub_rel_sks[exi], 'o', ms=2, alpha=0.1, color='peru', label='ELAIS')
plt.xlabel('i-band ccd offset (arcsec)')
# plt.ylabel('i-band sub skysig / exp skysig')
plt.show()

plt.figure(figsize=(12, 3))
plt.subplot(1, 3, 1)
plt.plot( sub_off[cxg], sub_rel_mzp[cxg], 'o', ms=2, alpha=0.1, color='darkgreen', label='COSMOS')
plt.plot( sub_off[exg], sub_rel_mzp[exg], 'o', ms=2, alpha=0.1, color='limegreen', label='ELAIS')
plt.xlabel('g-band ccd offset (arcsec)')
plt.ylabel('g-band sub zeropoint / exp zeropoint')
# plt.legend(loc='upper left')
plt.subplot(1, 3, 2)
plt.plot( sub_off[cxr], sub_rel_mzp[cxr], 'o', ms=2, alpha=0.1, color='red', label='COSMOS')
plt.plot( sub_off[exr], sub_rel_mzp[exr], 'o', ms=2, alpha=0.1, color='darkorange', label='ELAIS')
plt.xlabel('r-band ccd offset (arcsec)')
# plt.ylabel('r-band sub zeropoint / exp zeropoint')
# plt.legend(loc='upper left')
plt.subplot(1, 3, 3)
plt.plot( sub_off[cxi], sub_rel_mzp[cxi], 'o', ms=2, alpha=0.1, color='brown', label='COSMOS')
plt.plot( sub_off[exi], sub_rel_mzp[exi], 'o', ms=2, alpha=0.1, color='peru', label='ELAIS')
plt.xlabel('i-band ccd offset (arcsec)')
# plt.ylabel('i-band sub zeropoint / exp zeropoint')
plt.show()